In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


170500096/170498071 [==============================] - 9s 0us/step


In [0]:
# Image Augmentation

from keras.preprocessing.image import ImageDataGenerator
# define data preparation
shift = 0.2
datagen = ImageDataGenerator(width_shift_range=shift, height_shift_range=shift)
# fit parameters from data
datagen.fit(x_train)

datagen = ImageDataGenerator(zca_whitening=True)
# fit parameters from data
datagen.fit(x_train)

/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:645: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
#model.summary()

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 181s 4ms/step - loss: 1.7542 - acc: 0.3436 - val_loss: 1.7155 - val_acc: 0.3850
Epoch 2/50
19200/50000 [==========>...................] - ETA: 1:38 - loss: 1.4510 - acc: 0.4633

50000/50000 [==============================] - 169s 3ms/step - loss: 1.3831 - acc: 0.4900 - val_loss: 1.9960 - val_acc: 0.3847
Epoch 3/50
45568/50000 [==========================>...] - ETA: 14s - loss: 1.2245 - acc: 0.5542

50000/50000 [==============================] - 169s 3ms/step - loss: 1.2198 - acc: 0.5555 - val_loss: 1.3761 - val_acc: 0.5440
Epoch 4/50
50000/50000 [==============================] - 168s 3ms/step - loss: 1.1115 - acc: 0.5988 - val_loss: 1.6974 - val_acc: 0.4891
Epoch 5/50
 3328/50000 [>.............................] - ETA: 2:29 - loss: 1.0739 - acc: 0.6010

50000/50000 [==============================] - 167s 3ms/step - loss: 1.0410 - acc: 0.6230 - val_loss: 1.2187 - val_acc: 0.5950
Epoch 6/50
39552/50000 [======================>.......] - ETA: 33s - loss: 0.9887 - acc: 0.6441

50000/50000 [==============================] - 167s 3ms/step - loss: 0.9882 - acc: 0.6433 - val_loss: 1.1297 - val_acc: 0.6123
Epoch 7/50
50000/50000 [==============================] - 167s 3ms/step - loss: 0.9519 - acc: 0.6583 - val_loss: 0.9907 - val_acc: 0.6595
Epoch 8/50
 1920/50000 [>.............................] - ETA: 2:32 - loss: 0.8814 - acc: 0.6792

...

50000/50000 [==============================] - 166s 3ms/step - loss: 0.5315 - acc: 0.8128 - val_loss: 1.1091 - val_acc: 0.6998
Epoch 45/50
38656/50000 [======================>.......] - ETA: 35s - loss: 0.5238 - acc: 0.8143

50000/50000 [==============================] - 166s 3ms/step - loss: 0.5250 - acc: 0.8147 - val_loss: 0.7834 - val_acc: 0.7640
Epoch 46/50
50000/50000 [==============================] - 165s 3ms/step - loss: 0.5256 - acc: 0.8149 - val_loss: 0.9126 - val_acc: 0.7404
Epoch 47/50
 1536/50000 [..............................] - ETA: 2:31 - loss: 0.5192 - acc: 0.8197

50000/50000 [==============================] - 165s 3ms/step - loss: 0.5205 - acc: 0.8164 - val_loss: 1.2081 - val_acc: 0.6906
Epoch 48/50
38656/50000 [======================>.......] - ETA: 35s - loss: 0.5154 - acc: 0.8186

50000/50000 [==============================] - 165s 3ms/step - loss: 0.5179 - acc: 0.8178 - val_loss: 0.8760 - val_acc: 0.7328
Epoch 49/50
50000/50000 [==============================] - 165s 3ms/step - loss: 0.5150 - acc: 0.8187 - val_loss: 0.8978 - val_acc: 0.7494
Epoch 50/50
 1536/50000 [..............................] - ETA: 2:31 - loss: 0.4832 - acc: 0.8262

50000/50000 [==============================] - 165s 3ms/step - loss: 0.5099 - acc: 0.8206 - val_loss: 0.7418 - val_acc: 0.7735


In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 0.7418320245742798
Test accuracy: 0.7735


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')